In [2]:
using HVAC, ModelingToolkit, SciMLBase, OrdinaryDiffEq, ModelingToolkitStandardLibrary, ModelingToolkitStandardLibrary.Blocks
import HVAC: moistair, relativeHumidity_PTX, saturationPressure

In [3]:
# 1. Load the refrigerant: 
ref_name = "R32"
refrigerant = load_refrigerant(ref_name*".yaml")

Ref spline model

In [4]:
# 2. Choose initial conditions 

# 2.a. Refrigerant side (in & out): pressure, enthalpy, massflow rate, 
p_in, p_out = 8e5, 7.9e5
h_in, h_out = 230e3, 430e3
ref_mdot_start = 0.01

# 2.b. Air side (in): massflow rate, temperature, humidity
air_mdot_in = 0.01
air_T_in_start = 300.0        # Same as https://help.juliahub.com/hvac/stable/vapor_compression_cycle/
air_ϕ_in_Start = 0.5          # Same as https://help.juliahub.com/hvac/stable/vapor_compression_cycle/
air_p_in_start = 101325.0 

P_amb = 101325.0
air_p_out_start = P_amb - 1e-2     # But I think e-2 is too small to be considered, maybe 1e2       # Same as https://help.juliahub.com/hvac/stable/vapor_compression_cycle/
air_ϕ_out_Start = relativeHumidity_PTX(P_amb - 1e-2, from_degC(18), moistair.Xi_ref, moistair)      # Same as https://help.juliahub.com/hvac/stable/vapor_compression_cycle/
air_T_out_start = from_degC(18)                                                                     # Same as https://help.juliahub.com/hvac/stable/vapor_compression_cycle/

# 2.c. Geometery
nTube = 4
nSeg = 1
Lt = 20
Di = 2.54e-2
Do = Di + 0.5e-3

0.0259

In [5]:
# 3. Choose controller inputs: 
# Seems unnecessary for HX

In [6]:
# 4. Components

# Heat exchanger
HX_record = MERLTubeFinHEXRecord(; nTube, nSeg, Lt, Di, Do, 
    ref_model_structure = av_vb,           # @enum ModelStructure av_vb = 1 a_v_b = 2 a_vb = 3 av_b = 4 # in utils.jl
    ref_p_in_start = p_in,
    ref_p_out_start = p_out,
    ref_h_in_start = h_in,
    ref_h_out_start = h_out,
    ref_m_flow_start = ref_mdot_start,

    wall_T_in_start = from_degC(27),
    wall_T_out_start = from_degC(27),

    air_p_in_start,
    air_p_out_start,
    air_m_air_flow_start = air_mdot_in,
    air_T_in_start = air_T_in_start,
    air_T_out_start = air_T_out_start,
    air_Xi_in_start = moistair.Xi_ref,
    air_Xi_out_start = moistair.Xi_ref,
    air_Q_start = 50 
    )

@named heatExchanger = TubeFinHEX(HX_record)

Model heatExchanger with 491 (539) equations
Unknowns (539):
  dP_ref_hx(t): Tube fin HX - refrigerant side pressure drop [Pa]
  Q_tot(t): Tube fin HX - Air side heat load [W]
  Qsen_tot(t): Tube fin HX - Air side sensible heat load [W]
  Qlat_tot(t): Tube fin HX - Air side latent heat load [W]
  Mtot(t): Tube fin HX - Total refrigerant charge [kg]
  Tair_out(t): Tube fin HX - Air side exit temperature [K]
⋮
Parameters (115):
  circuit₊fluidVolumes[1] [defaults to circuit₊crossAreas[1]*circuit₊lengths[1]]: Volume [m³] of corresponding control volume. Determined in inheriting class
  circuit₊fluidVolumes[2] [defaults to circuit₊crossAreas[2]*circuit₊lengths[2]]: Volume [m³] of corresponding control volume. Determined in inheriting class
  circuit₊fluidVolumes[3] [defaults to circuit₊crossAreas[3]*circuit₊lengths[3]]: Volume [m³] of corresponding control volume. Determined in inheriting class
  circuit₊fluidVolumes[4] [defaults to circuit₊crossAreas[4]*circuit₊lengths[4]]: Volume [m³] of

In [10]:
# Ref side:
@named refSource = MassFlowSource_h(m_flow_in = ref_mdot_start, h_in = h_in, P_start = p_in)
@named refSink = Boundary_Ph(P_in = p_out, h_in = h_out, m_flow_start = ref_mdot_start)

# Air side: 
@named AirSource = MassFlowSource_Tϕ(; T_in = air_T_in_start, ϕ_in = air_ϕ_in_Start, m_flow_in = air_mdot_in, P_start = air_p_in_start)
@named AirSink = Boundary_PTϕ(; P_in = air_p_out_start, T_in = air_T_out_start, ϕ_in = air_ϕ_in_Start, m_flow_start = air_mdot_in)
     ## 出口的P T ϕ 会变吗？

Model AirSink with 15 (16) equations
Unknowns (16):
  port1₊P(t)
  port1₊m_flow(t)
  port1₊h_outflow(t)
  port1₊Xi_outflow(t)
  medium₊P(t)
  medium₊T(t)
⋮
Parameters (3):
  P_in [defaults to 101325.0]: Moist air Source/Sink - Fixed pressure
  T_in [defaults to 291.15]: Moist air Source/Sink - Fixed temperature
  ϕ_in [defaults to 0.5]: Moist air Source/Sink - Fixed moisture fraction

In [11]:
# 5. System

# Equations
eqns = [
    # Ref
    connect(refSource.port, heatExchanger.refPort_a)
    connect(heatExchanger.refPort_b, refSink.port1)
    
    # Air
    connect(AirSource.port, heatExchanger.airPort_a)
    connect(heatExchanger.airPort_b, AirSink.port1)
]

systems = [heatExchanger, refSource, refSink, AirSource, AirSink]

Number of systems: 5
heatExchanger: 539 states, 491 equations, 48 connection equations
refSource: 4 states, 3 equations, 1 connection equations
refSink: 21 states, 20 equations, 1 connection equations
AirSource: 7 states, 6 equations, 1 connection equations
AirSink: 16 states, 15 equations, 1 connection equations
Total: 587 states, 535 equations, 52 connection equations


In [12]:
# Define the ODE system: 
@named sys = ODESystem(eqns, t; systems)

# Simplificaiton
sysRed = structural_simplify(sys)
sysRed_debug = debug_sysRed(sysRed)

(eqns_reduced = Equation[Differential(t)(heatExchanger₊circuit₊medium1₊P(t)) ~ ((HVAC.semilinear(heatExchanger₊circuit₊flowModel₊m_flow_nominal*NaNMath.pow(0.0025000000000000005 + ((heatExchanger₊circuit₊medium2₊P(t) - heatExchanger₊circuit₊medium1₊P(t))^2) / (heatExchanger₊circuit₊flowModel₊dP_nominal^2), (1//2)*(-3 + 1.0 / heatExchanger₊circuit₊flowModel₊dP_exp))*NaNMath.pow((-heatExchanger₊circuit₊medium2₊P(t) + heatExchanger₊circuit₊medium1₊P(t)) / heatExchanger₊circuit₊flowModel₊dP_nominal, 3)*NaNMath.pow(1.0 / heatExchanger₊circuit₊flowModel₊K_dP, 1.0 / heatExchanger₊circuit₊flowModel₊dP_exp), heatExchanger₊circuit₊medium1₊h(t), heatExchanger₊circuit₊medium2₊h(t)) - HVAC.semilinear(refSource₊m_flow_in, refSource₊h_in, heatExchanger₊circuit₊medium1₊h(t)) - heatExchanger₊circuit₊heatTransfer₊surfaceAreas[1]*((heatExchanger₊wall₊T(t))[1] - RefpropSplines.Temperature_Ph(heatExchanger₊circuit₊medium1₊P(t), heatExchanger₊circuit₊medium1₊h(t), Ref spline model))*ifelse(HVAC.splice_funct

In [13]:
# 调试检查，检查方程的左侧是否是一个连接方程（Connection），并打印出方程右侧（eq.rhs）系统中涉及到的两个系统的名称

connect_prints(eq) = begin
    if eq.lhs isa Connection
        println(eq.rhs.systems[1].name, " , ", eq.rhs.systems[2].name)
    end
end

connect_prints.(eqns)

refSource₊port , heatExchanger₊refPort_a
heatExchanger₊refPort_b , refSink₊port1

4-element Vector{Nothing}:
 nothing
 nothing
 nothing
 nothing


AirSource₊port , heatExchanger₊airPort_a
heatExchanger₊airPort_b , AirSink₊port1


In [14]:
# Define the ODE problem and then DAE problem

tspan = (0.0, 1400.0)

prob = ODEProblem(sysRed, sysRed_debug.defs_set, tspan; guesses = ModelingToolkit.get_guesses(sysRed))

prob = init_DAE(prob, loops = 2, verbose = true, sparsity = false)

(blk, alge_to_all[p[blk]], alge_to_all[q[blk]], length(blk)) = (1:21, [12, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33], [12, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33], 21)
norm(out) / norm(nlu) = 0.07935366760009582
(sol.residual_norm, blk, alge_to_all[p[blk]], alge_to_all[q[blk]], length(blk)) = (27.486056536334218, 22:22, [15], [33], 1)
(blk, alge_to_all[p[blk]], alge_to_all[q[blk]], length(blk)) = (21:21, [33], [28], 1)
(blk, alge_to_all[p[blk]], alge_to_all[q[blk]], length(blk)) = (20:20, [32], [23], 1)
(blk, alge_to_all[p[blk]], alge_to_all[q[blk]], length(blk)) = (19:19, [31], [18], 1)
(blk, alge_to_all[p[blk]], alge_to_all[q[blk]], length(blk)) = (18:18, [29], [14], 1)
(blk, alge_to_all[p[blk]], alge_to_all[q[blk]], length(blk)) = (17:17, [26], [31], 1)
(blk, alge_to_all[p[blk]], alge_to_all[q[blk]], length(blk)) = (15:16, [25, 27], [29, 30], 2)
(blk, alge_to_all[p[blk]], alge_to_all[q[blk]], length(blk)) 

ODEProblem with uType Vector{Float64} and tType Float64. In-place: true
timespan: (0.0, 1400.0)
u0: 34-element Vector{Float64}:
 800000.0
 230000.0
 796666.6666666666
 296666.6666666667
 793333.3333333334
 363333.3333333333
 430000.0
    300.15
    300.15
    300.15
    300.15
    291.1482369544362
     -0.25333572499168544
      ⋮
    299.9981833636643
      0.009993944545547606
  52244.38775957381
      0.009993944545547606
    299.9981833636643
    299.9981833636643
   1014.3225000000001
      0.009993944545547606
  52244.38775957381
    299.9981833636643
    NaN
      0.006363461991259625

In [15]:
# Solve

sol = solve(prob, Rodas4(autodiff = false))

LoadError: MethodError: no method matching Float64(::ForwardDiff.Dual{ForwardDiff.Tag{SciMLBase.JacobianWrapper{true, NonlinearFunction{true, SciMLBase.FullSpecialize, OrdinaryDiffEqNonlinearSolve.var"#51#54"{ODEFunction{true, SciMLBase.AutoSpecialize, ModelingToolkit.var"#f#781"{RuntimeGeneratedFunctions.RuntimeGeneratedFunction{(:ˍ₋arg1, :ˍ₋arg2, :t), ModelingToolkit.var"#_RGF_ModTag", ModelingToolkit.var"#_RGF_ModTag", (0xddef88ae, 0xfbd167bf, 0xc8eec8fb, 0xcf4895df, 0xbd916d59), Nothing}, RuntimeGeneratedFunctions.RuntimeGeneratedFunction{(:ˍ₋out, :ˍ₋arg1, :ˍ₋arg2, :t), ModelingToolkit.var"#_RGF_ModTag", ModelingToolkit.var"#_RGF_ModTag", (0x25c28e4d, 0x06a3583c, 0x61c0982b, 0xb5b08642, 0x05d82d83), Nothing}}, Matrix{Float64}, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, ModelingToolkit.ObservedFunctionCache{ODESystem}, Nothing, ODESystem, Nothing, Nothing}, UnionAll, Bool, Vector{Float64}, Vector{Float64}, Vector{Float64}, Vect

In [ ]:
#=
# 蒸发器中连接的空气流量守恒的等式。引入一个非常小的校正因子（1e-2），用于调试或者改善仿真中的数值稳定性。
old_eq = 0 ~ evaporator.air_merger.port_b.m_flow - evaporator.airPort_b.m_flow
new_eq = 0 ~ evaporator.air_merger.port_b.m_flow - evaporator.airPort_b.m_flow -1e-2

sysRed = structural_simplify(sys; substitute_eqs = true, old_eq, new_eq)
=#